# Importing and Tidying data

In [24]:
import pandas as pd
import numpy as np

### Modifying HTML code

The given data is in a large table on the Winston's Lab website. This table is almost formatted perfectly, but there are a few columns that have nothing but pictures. I need to somehow convert these images to a string that I can use.

This function will basically replace the image code with the title of the image.

In [64]:
#convert img entries into words to be used
def fix_html_img(in_file_name, out_file_name):
    with open(in_file_name, 'r') as in_file:
        with open(out_file_name, 'w') as out_file:
            for line in in_file:
                line = line.replace('style="display: none;"', '')
                string = ''
                if "img" in line and "title" in line:
                    line_sep = line.split('\"')
                    if "class" in line:
                        string = string + ', ' + line_sep[1]
                        string = string + ' ' + line_sep[5]
                    else:
                        string = string + ', ' + line_sep[3]
                else:
                    string = line
                out_file.write(string)
            

Now I need to test to make sure that it works as intended.

In [65]:
#test file 1
test_in = 'untitled3.txt'
test_out = 'test.out'
fix_html_img(test_in, test_out)
with open(test_out, 'r') as file:
    lines = file.readlines()
    assert(len(lines) == 4)
    assert(lines[0] == '<td class="table-count table-basic">\n')
    assert(lines[1] == 'B</td>\n')
    assert(lines[2] == '<td class="table-death-sequence table-basic">\n')
    assert(lines[3] == ', red Winston, blue Zenyatta, blue Winston, blue Mccree, blue D.Va, blue Genji, blue Mccree')

In [66]:
#test file 2
test_in = 'untitled4.txt'
test_out = 'test2.out'
fix_html_img(test_in, test_out)
with open(test_out, 'r') as file:
    lines = file.readlines()
    assert(len(lines) == 5)
    assert(lines[0] == '</td>\n')
    assert(lines[1] == '<td class="table-heroes ults table-adv" style="display: none;">\n')
    assert(lines[2] == ', Mccree, Mercy</td>\n')
    assert(lines[3] == '<td class="table-heroes ults table-adv" style="display: none;">\n')
    assert(lines[4] == ', Mercy, Tracer, Genji</td>')

AssertionError: 

Ready for the real transformation.

In [67]:
real_in = 'table.txt'
real_out = 'table_out.txt'
fix_html_img(real_in, real_out)

## Importing Data

Luckily, Pandas includes a function that will easily translate HTML code to a dataframe.

In [56]:
txt = '''
<table class="table table-striped tablesorter box-shadow tablesorter-default" style="font-size: 13px;" role="grid">
    <thead>
        <tr role="row" class="tablesorter-headerRow">
            <th class="table-name tablesorter-header tablesorter-headerUnSorted" data-column="0" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Map: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Map</div>
            </th>
            <th class="table-name tablesorter-header tablesorter-headerUnSorted" data-column="1" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Roundtype: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Roundtype</div>
            </th>
            <th class="table-teamlogo tablesorter-header tablesorter-headerUnSorted" data-column="2" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Blue Team: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Blue Team</div>
            </th>

            <th class="table-teamlogo tablesorter-header tablesorter-headerUnSorted" data-column="3" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Red Team: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Red Team</div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="4" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="time?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="timestamp of when fight began">
                    time<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="5" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="len?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="length of the fight in seconds">
                    len<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="6" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="KB?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="Kills Blue: amount of kills the blue team made">
                    KB<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="7" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="KR?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="Kills Red: amount of kills the red team made">
                    KR<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="8" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="UB?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="Ults Blue: amount of ults the blue team used">
                    UB<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="9" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="UR?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="Ults Red: amount of ults the red team used">
                    UR<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

            <th class="table-count table-basic tablesorter-header tablesorter-headerUnSorted" data-column="10" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="FB?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="First Blood: B=blue team, R=red team">
                    FB<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>

             <th class="table-death-sequence table-basic tablesorter-header tablesorter-headerUnSorted" data-column="11" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Death Sequence?: No sort applied, activate to apply an ascending sort">
                <div class="tablesorter-header-inner">
                    <abbr title="blue border=player from blue team">
                    Death Sequence<span style="font-size: 9px;vertical-align: super;color: #666;">
                    ?</span>
                    </abbr>
                </div>
            </th>
            
            <th class="table-name tablesorter-header tablesorter-headerUnSorted" data-column="1" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Roundtype: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Roundtype</div>
            </th>

            <th class="table-heroes ults table-adv tablesorter-header tablesorter-headerUnSorted" data-column="12" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Ults Blue: No sort applied, activate to apply an ascending sort" style="user-select: none;">
                <div class="tablesorter-header-inner">
                    Ults Blue</div>
            </th>

            <th class="table-heroes ults table-adv tablesorter-header tablesorter-headerUnSorted" data-column="13" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Ults Red: No sort applied, activate to apply an ascending sort" style="display: none;">
                <div class="tablesorter-header-inner">
                    Ults Red</div>
            </th>

            <th class="table-heroes table-adv tablesorter-header tablesorter-headerUnSorted" data-column="14" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Teamcomp Blue: No sort applied, activate to apply an ascending sort" style="display: none;">
                <div class="tablesorter-header-inner">
                    Teamcomp Blue</div>
            </th>

            <th class="table-heroes table-adv tablesorter-header tablesorter-headerUnSorted" data-column="15" tabindex="0" scope="col" role="columnheader" aria-disabled="false" unselectable="on" aria-sort="none" aria-label="Teamcomp Red: No sort applied, activate to apply an ascending sort" style="display: none;">
                <div class="tablesorter-header-inner">
                    Teamcomp Red</div>
            </th>
        </tr>
    </thead>

<tbody id="statsTableBody" aria-live="polite" aria-relevant="all">
<tr class="">
<td class="table-name">
Dorado</td>
<td class="table-name">
Attack</td>
<td class="table-teamlogo">
<img src="/pics/teams/shock.png">
 <abbr title="San Francisco Shock">
SFS</abbr>
</td>
<td class="table-teamlogo">
<img src="/pics/teams/la_valiant2.png">
 <abbr title="Los Angeles Valiant">
VAL</abbr>
</td>
<td class="table-count table-basic">
0:29</td>
<td class="table-count table-basic">
14</td>
<td class="table-count table-basic">
5</td>
<td class="table-count table-basic">
0</td>
<td class="table-count table-basic">
0</td>
<td class="table-count table-basic">
0</td>
<td class="table-count table-basic">
B</td>
<td class="table-death-sequence table-basic">
<img class="red" src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img class="red" src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img class="red" src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img class="red" src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img class="red" src="/pics/heroes/tracer-icon-portrait.png" title="Tracer">
</td>
<td class="table-heroes ults table-adv" style="display: none;">
</td>
<td class="table-heroes ults table-adv" style="display: none;">
</td>
<td class="table-heroes table-adv" style="display: none;">
<img src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img src="/pics/heroes/mccree-icon-portrait.png" title="Mccree">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img src="/pics/heroes/winston-icon-portrait.png" title="Winston">
</td>
<td class="table-heroes table-adv" style="display: none;">
<img src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img src="/pics/heroes/tracer-icon-portrait.png" title="Tracer">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img src="/pics/heroes/winston-icon-portrait.png" title="Winston">
</td>
</tr>
<tr class="">
<td class="table-name">
Dorado</td>
<td class="table-name">
Attack</td>
<td class="table-teamlogo">
<img src="/pics/teams/shock.png">
 <abbr title="San Francisco Shock">
SFS</abbr>
</td>
<td class="table-teamlogo">
<img src="/pics/teams/la_valiant2.png">
 <abbr title="Los Angeles Valiant">
VAL</abbr>
</td>
<td class="table-count table-basic">
1:05</td>
<td class="table-count table-basic">
24</td>
<td class="table-count table-basic">
1</td>
<td class="table-count table-basic">
6</td>
<td class="table-count table-basic">
2</td>
<td class="table-count table-basic">
3</td>
<td class="table-count table-basic">
B</td>
<td class="table-death-sequence table-basic">
<img class="red" src="/pics/heroes/winston-icon-portrait.png" title="Winston">
<img class="blue" src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img class="blue" src="/pics/heroes/winston-icon-portrait.png" title="Winston">
<img class="blue" src="/pics/heroes/mccree-icon-portrait.png" title="Mccree">
<img class="blue" src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img class="blue" src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img class="blue" src="/pics/heroes/mccree-icon-portrait.png" title="Mccree">
</td>
<td class="table-heroes ults table-adv" style="display: none;">
<img src="/pics/heroes/mccree-icon-portrait.png" title="Mccree">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
</td>
<td class="table-heroes ults table-adv" style="display: none;">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img src="/pics/heroes/tracer-icon-portrait.png" title="Tracer">
<img src="/pics/heroes/genji-icon-portrait.png" title="Genji">
</td>
<td class="table-heroes table-adv" style="display: none;">
<img src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img src="/pics/heroes/mccree-icon-portrait.png" title="Mccree">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img src="/pics/heroes/winston-icon-portrait.png" title="Winston">
</td>
<td class="table-heroes table-adv" style="display: none;">
<img src="/pics/heroes/genji-icon-portrait.png" title="Genji">
<img src="/pics/heroes/tracer-icon-portrait.png" title="Tracer">
<img src="/pics/heroes/mercy-icon-portrait.png" title="Mercy">
<img src="/pics/heroes/zenyatta-icon-portrait.png" title="Zenyatta">
<img src="/pics/heroes/dva-icon-portrait.png" title="D.Va">
<img src="/pics/heroes/winston-icon-portrait.png" title="Winston">
</td>
</tr>
<tr class="">
<td class="table-name">
Dorado</td>
<td class="table-name">
Attack</td>
                                        '''

In [57]:
from IPython.display import display_html
display_html(txt, raw=True)

Map 
 
 
 
 Roundtype 
 
 
 
 Blue Team 
 

 
 
 Red Team 
 

 
 
 
 time 
 ? 
 
 
 

 
 
 
 len 
 ? 
 
 
 

 
 
 
 KB 
 ? 
 
 
 

 
 
 
 KR 
 ? 
 
 
 

 
 
 
 UB 
 ? 
 
 
 

 
 
 
 UR 
 ? 
 
 
 

 
 
 
 FB 
 ? 
 
 
 

 
 
 
 Death Sequence 
 ? 
 
 
 
 
 
 
 Roundtype 
 

 
 
 Ults Blue 
 

 
 
 Ults Red 
 

 
 
 Teamcomp Blue 
 

 
 
 Teamcomp Red 
 
 
 

 
 
 
Dorado 
 
Attack 
 
 
 
SFS 
 
 
 
 
VAL 
 
 
0:29 
 
14 
 
5 
 
0 
 
0 
 
0 
 
B 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Dorado 
 
Attack 
 
 
 
SFS 
 
 
 
 
VAL 
 
 
1:05 
 
24 
 
1 
 
6 
 
2 
 
3 
 
B 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Dorado 
 
Attack

In [42]:
df = pd.read_html('table.txt',header=0)[0]
print(df.columns)

Index(['Map', 'Roundtype', 'Blue Team', 'Red Team', 'time ?', 'len ?', 'KB ?',
       'KR ?', 'UB ?', 'UR ?', 'FB ?', 'Death Sequence ?'],
      dtype='object')


In [35]:
print(df[1])

IndexError: list index out of range

In [36]:
print(df[2])

IndexError: list index out of range

In [68]:
df = pd.read_html('table_out.txt')[0]
df = df.fillna('')

In [70]:
df.head()

,Map,Roundtype,Blue Team,Red Team,time ?,len ?,KB ?,KR ?,UB ?,UR ?,FB ?,Death Sequence ?,Ults Blue,Ults Red,Teamcomp Blue,Teamcomp Red
0,Dorado,Attack,SFS,VAL,0:29,14,5,0,0,0,B,", red Mercy, red Zenyatta, red Genji, red D.Va...",,,", Genji, Mccree, Mercy, Zenyatta, D.Va, Winston",", Genji, Tracer, Mercy, Zenyatta, D.Va, Winston"
1,Dorado,Attack,SFS,VAL,1:05,24,1,6,2,3,B,", red Winston, blue Zenyatta, blue Winston, bl...",", Mccree, Mercy",", Mercy, Tracer, Genji",", Genji, Mccree, Mercy, Zenyatta, D.Va, Winston",", Genji, Tracer, Mercy, Zenyatta, D.Va, Winston"
2,Dorado,Attack,SFS,VAL,1:51,7,0,1,1,0,R,", blue Winston",", Mercy",,", Genji, Mccree, Mercy, Zenyatta, D.Va, Winston",", Genji, Tracer, Mercy, Zenyatta, D.Va, Winston"
3,Dorado,Attack,SFS,VAL,2:14,28,7,2,3,3,B,", red Genji, blue Genji, red Tracer, red D.Va,...",", Zenyatta, Genji, Mccree",", Tracer, Zenyatta, Winston",", Genji, Mccree, Mercy, Zenyatta, D.Va, Winston",", Genji, Tracer, Mercy, Zenyatta, D.Va, Winston"
4,Dorado,Attack,SFS,VAL,3:24,17,4,2,3,1,R,", blue Zenyatta, red Winston, red Mercy, red D...",", Mercy, Mccree, Winston",", Genji",", Mccree, Tracer, Mercy, Zenyatta, D.Va, Winston",", Widowmaker, Genji, Mercy, Zenyatta, D.Va, Wi..."


In [71]:
#Change columns to a more usable format
df.columns = ['Map', 'Roundtype', 'Blue_Team', 'Red_Team', 'time', 'len', 'KB',
       'KR', 'UB', 'UR', 'FB', 'Death_Sequence', 'Ults_Blue', 'Ults_Red',
       'Teamcomp_Blue', 'Teamcomp_Red']

In [72]:
assert(list(df['Map'].unique()) == ['Dorado', 'Temple of Anubis', 'Ilios', 'Numbani', 'Eichenwalde', 'Junkertown', 'Oasis', 'Horizon Lunar Colony', 'Lijiang Tower'])
assert(list(df['Roundtype'].unique()) == ['Attack', 'Defense', 'Lighthouse', 'Ruins', 'Well', 'University', 'Gardens', 'City Center', 'Control Center', 'Night Market', 'Garden'])
assert(list(df['Blue_Team'].unique()) == ['SFS', 'SHD', 'DAL','LDN', 'PHI', 'BOS','VAL', 'FLA', 'NYE', 'SEO', 'HOU', 'GLA'])
assert(list(df['Red_Team'].unique()) == ['VAL', 'GLA', 'SEO', 'FLA','HOU', 'NYE', 'DAL', 'BOS', 'SHD', 'PHI', 'LDN', 'SFS'])

In [73]:
df['Death_Sequence'] = df['Death_Sequence'].str[2:].str.split(', ')
df['Teamcomp_Blue'] = df['Teamcomp_Blue'].str[2:].str.split(', ')
df['Teamcomp_Red'] = df['Teamcomp_Red'].str[2:].str.split(', ')
df['Ults_Blue'] = df['Ults_Blue'].str[2:].str.split(', ')
df['Ults_Red'] = df['Ults_Red'].str[2:].str.split(', ')

Unfortunately, the dataframe did not record the "winner" of a fight. I will define the winner as the team who has gotten the most kills.

In [74]:
def findWinner(x):
    if x['KR'] > x['KB']:
        return 'Red'
    elif x['KR'] < x['KB']:
        return 'Blue'
    else:
        return 'Tie'
    
df['Winner'] = df.apply(findWinner, axis=1)

## Tidying Data

The dataset we have is not the most useful. Specifically, the Death Sequence and Ultimates Used columns are a string of multiple things used. The main goal now is to change those into usable tidy columns.

In [75]:
#Make a tidy column from the death sequence
df['First_kill'] = df['Death_Sequence'].str[0].str.split().str[1]

In [76]:
assert(list(df['Winner'].unique()) == ['Blue', 'Red', 'Tie'])

First, change each string entry into a list.

In [77]:
for i in range(6):
    strR = 'Red' + str(i)
    strB = 'Blue' + str(i)
    df[strR] = df['Winner']
    df[strB] = df['Winner']
    for j in range(len(df)):
        df.loc[j, strR] = df['Teamcomp_Red'][j][i]
        df.loc[j, strB] = df['Teamcomp_Blue'][j][i]

In [78]:
cols = df.columns
cols1 = np.append(cols[:-11], [cols[-10], cols[-8], cols[-6], cols[-4], cols[-2]])
cols1 = np.append(cols1, [cols[-11], cols[-9], cols[-7], cols[-5] , cols[-3] , cols[-1]])

df = df[cols1]

Make columns for Red/Blue hero 1 - 6

In [79]:
#Takes a while to run
red_comp = pd.DataFrame([])
blue_comp = pd.DataFrame([])
for i in range(len(df)):
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red0']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red1']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red2']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red3']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red4']] = 1
    red_comp.loc[i, 'Red_' + df.loc[i, 'Red5']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue0']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue1']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue2']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue3']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue4']] = 1
    blue_comp.loc[i, 'Blue_' + df.loc[i, 'Blue5']] = 1
red_comp = red_comp.fillna(0)
blue_comp = blue_comp.fillna(0)
df = pd.merge(df, red_comp, left_index=True, right_index=True)
df = pd.merge(df, blue_comp, left_index=True, right_index=True)

Future proofing: I know that I will be using these columns to make a model, so I must convert the columns to numbers.

In [80]:
#Make the winner values easier to understand when we get dummies later
df['Winner'] = df['Winner'].map({'Blue':'Blue_Winner', 'Red':'Red_Winner', 'Tie':'Tie'})
df['FB'] = df['FB'].map({'B':1, 'R':0})

In [81]:
map_series = pd.get_dummies(df['Map'])
df = pd.merge(df, map_series, left_index=True, right_index=True)
round_series = pd.get_dummies(df['Roundtype'])
df = pd.merge(df, round_series, left_index=True, right_index=True)
win_series = pd.get_dummies(df['Winner'])
df = pd.merge(df, win_series, left_index=True, right_index=True)
blue_series = pd.get_dummies(df['Blue_Team'], prefix='Blue')
df = pd.merge(df, blue_series, left_index=True, right_index=True)
red_series = pd.get_dummies(df['Red_Team'], prefix='Red')
df = pd.merge(df, red_series, left_index=True, right_index=True)


I have to drop the columns that we updated now.

In [82]:
df = df.drop(columns=['Death_Sequence', 'Ults_Blue',
 'Ults_Red', 'Teamcomp_Blue',
 'Teamcomp_Red',  'Red0',
 'Red1',
 'Red2',
 'Red3',
 'Red4',
 'Red5',
 'Blue0',
 'Blue1',
 'Blue2',
 'Blue3',
 'Blue4',
 'Blue5'])

In [83]:
df.head()

,Map,Roundtype,Blue_Team,Red_Team,time,len,KB,KR,UB,UR,...,Red_FLA,Red_GLA,Red_HOU,Red_LDN,Red_NYE,Red_PHI,Red_SEO,Red_SFS,Red_SHD,Red_VAL
0,Dorado,Attack,SFS,VAL,0:29,14,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Dorado,Attack,SFS,VAL,1:05,24,1,6,2,3,...,0,0,0,0,0,0,0,0,0,1
2,Dorado,Attack,SFS,VAL,1:51,7,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,Dorado,Attack,SFS,VAL,2:14,28,7,2,3,3,...,0,0,0,0,0,0,0,0,0,1
4,Dorado,Attack,SFS,VAL,3:24,17,4,2,3,1,...,0,0,0,0,0,0,0,0,0,1


## Save the Data!

In [84]:
df.to_csv('OverwatchFightsData.csv')